In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer, GenerationConfig
import evaluate
import pandas as pd
import numpy as np

import torch
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device= "cpu"
device

'cpu'

In [2]:
tokenizer = AutoTokenizer.from_pretrained('t5-large')
model = AutoModelForSeq2SeqLM.from_pretrained('t5-large')

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files=r"F:\linkedin\job_data.csv")
dataset = dataset.remove_columns(column_names=['Unnamed: 0.1','Unnamed: 0'])
dataset = dataset['train'].train_test_split(0.1)

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Description', 'Company Name'],
        num_rows: 4517
    })
    test: Dataset({
        features: ['Description', 'Company Name'],
        num_rows: 502
    })
})

In [5]:
def tokenizing_data(data):
    
    inputs = tokenizer(
        data['Description'],
        max_length=812,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

    outputs = tokenizer(
        data['Company Name'],
        max_length=15,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

    inputs['labels'] = outputs['input_ids']

    return inputs

In [6]:
training_data = dataset['train'].map(tokenizing_data,batched=True,batch_size=1)
training_data = training_data.remove_columns(column_names=['Description','Company Name'])
print(training_data)

testing_data = dataset['test'].map(tokenizing_data,batched=True,batch_size=1)
testing_data = testing_data.remove_columns(column_names=['Description','Company Name'])
print(testing_data)

Map:   0%|          | 0/4517 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 4517
})


Map:   0%|          | 0/502 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 502
})


In [7]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))


trainable model parameters: 737668096
all model parameters: 737668096
percentage of trainable model parameters: 100.00%


In [8]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=2,
    lora_alpha=32,
    lora_dropout=0.1,
)

peft_model_train = get_peft_model(model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model_train))

trainable model parameters: 589824
all model parameters: 738257920
percentage of trainable model parameters: 0.08%


In [9]:
output_dir = "./output_folder"

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=10,
    no_cuda=True
)

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1376: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [10]:
peft_trainer = Trainer(
    model=peft_model_train, 
    args=peft_training_args,
    train_dataset = training_data,
    eval_dataset=testing_data,
    )

peft_trainer.train()

  0%|          | 0/5650 [00:00<?, ?it/s]

Checkpoint destination directory ./output_folder\checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'loss': 1.026, 'grad_norm': 19.421674728393555, 'learning_rate': 0.000911504424778761, 'epoch': 0.88}


Checkpoint destination directory ./output_folder\checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'loss': 0.5902, 'grad_norm': 4.374779224395752, 'learning_rate': 0.0008230088495575221, 'epoch': 1.77}


...

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'loss': 0.3475, 'grad_norm': 2.023759126663208, 'learning_rate': 2.6548672566371683e-05, 'epoch': 9.73}


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'train_runtime': 248044.1537, 'train_samples_per_second': 0.182, 'train_steps_per_second': 0.023, 'train_loss': 0.5053076077351528, 'epoch': 10.0}


TrainOutput(global_step=5650, training_loss=0.5053076077351528, metrics={'train_runtime': 248044.1537, 'train_samples_per_second': 0.182, 'train_steps_per_second': 0.023, 'train_loss': 0.5053076077351528, 'epoch': 10.0})

In [12]:
peft_trainer.model.save_pretrained("./output_t5_model")
tokenizer.save_pretrained("./output_t5_model")

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('./output_t5_model\\tokenizer_config.json',
 './output_t5_model\\special_tokens_map.json',
 './output_t5_model\\tokenizer.json')

In [13]:
from peft import PeftModel

In [14]:
peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("t5-large")
peft_model = PeftModel.from_pretrained(peft_model_base, "./output_t5_model", is_trainable=False)

In [18]:
trail_inputs = tokenizer.encode("job title aiml engineerlocation 100% remote quarterly travel company hqcompensation based experience qualifications subject base salary bonusabout east 57th street partners internationaleast 57th street partners international partners companies forefront technology provide innovative solutions ai machine learning executive direct hire search practice place technical leaders drive cuttingedge development optimizationposition overviewour client seeking aiml engineer join remote team lead development machine learning models enhance product offerings aiml engineer work closely data scientists software developers build train deploy models solve realworld problems position requires quarterly travel company’s headquarters key strategic discussions collaborationkey responsibilitiesdesign develop implement ai machine learning algorithmscollaborate crossfunctional teams integrate ai solutions existing productsuse advanced statistical techniques identify patterns build predictive modelstrain machine learning models optimize performance monitor outcomesstay updated latest advancements aiml technologies implement best practiceswork large datasets develop scalable ai solutionsqualificationsbachelor’s master’s degree computer science ai machine learning related fieldproven experience aiml development including model building deploymentproficiency python tensorflow pytorch aiml frameworksstrong problemsolving analytical skillsability work independently remote setting collaborating global teamscompensation benefitscompetitive compensation package based experience qualifications including base salary bonusaccess cuttingedge aiml projects professional development opportunities")

In [52]:
input_ids = tokenizer(dataset['train']['Description'][2004], return_tensors="pt").input_ids
peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

'Accenture in India'

In [53]:
dataset['train']['Description'][2004]

'project role ai ml engineer project role description develops applications systems utilize ai tools cloud ai services proper cloud onprem application pipeline production ready quality able apply genai models part solution could also include limited deep learning neural networks chatbots image processing must skills google cloud machine learning services maestro good skills na minimum 5 years experience required educational qualification 15 years full time education summary aiml engineer develop applications systems utilize ai tools cloud ai services responsible creating proper cloud onprem application pipeline productionready quality additionally apply genai models part solution may include deep learning neural networks chatbots image processing roles & responsibilities expected sme collaborate manage team perform responsible team decisions engage multiple teams contribute key decisions provide solutions problems immediate team across multiple teams develop applications systems utiliz

In [54]:
dataset['train']['Company Name'][2004]

'Accenture in India'

In [64]:
input_ids = tokenizer("""We are looking for an experienced developer to join our team. You must have 1 to 3 years of work experience in ML/AI - Computer Vision and should have delivered at least 2 Deep Learning projects.

Who am I?

Have delivered at least 2 Deep Learning projects.
A self-learner and result-oriented individual with an ability to work in teams and help scale the teams.
Have a delivery and client-oriented approach.
Have good written and verbal communication skills.
Have a problem-solving approach and can work independently.
Have deep knowledge of math, probability, statistics and algorithms.
Passion for continuing to learn state-of-the-art techniques in ML/Data science.
Any Science or Engineering graduate
What should my technical skills be?

Have proficient knowledge of deep learning frameworks such as PyTorch, TensorFlow or Keras.
Hands-on experience with Python and libraries like OpenCV and Pillow.
Familiarity with object detection algorithms like FasterRCNN, Yolo, SSD.
Knowledge of frameworks like Detectron2
Working knowledge of how Neural Network works and various pre-trained models.
Knowledge of models like XGBOOST.
Experience in Model Conversion (ONNX).
Expertise in visualizing and manipulating big datasets.
Understanding of data structures, data modelling and software architecture.
Experience in working on C++ is an added advantage
What will I do?

Research machine learning algorithms and implement by tailoring to particular business needs.
Work on training and identifying Products and Objects using Computer Vision.
Provide analytical support to improve quality and standard work results.
Deploy solutions involving Image processing.
Pipeline deployment of Models and their ensembling.
Why should I join Big Rattle?

Big Rattle Technologies specializes into development of Mobile and Web applications. Our clients include Fortune 5000 companies. Over the past 12 years, we have delivered multiple projects for clients from various industries like FMCG, Restaurants, Banking and Finance, Automobiles, Ecommerce, Television and Broadcast Agencies. Big Rattle Technologies Private Limited is ISO27001 Certified

You will get:

End-to-end exposure in software design and development of products, web applications and mobile applications.
Opportunities to participate in the decision-making process and take leadership roles.
Opportunities to build and render exceptional products and services.
Employee Benefits include:

Healthcare coverage
Certification reimbursement
5-day hybrid work week
Laptop
Social media presence:

Website: www.bigrattle.com
Glassdoor: https://www.glassdoor.co.in/Reviews/Big-Rattle-Technologies-Reviews-E597660.htm
LinkedIn: https://www.linkedin.com/company/big-rattle-technologies
Facebook: https://www.facebook.com/bigrattle
Consistent Clutch leader for 3 consecutive years 2021 | 2020 | 2019.
Featured on Mobile App Daily among the Top 10 App development companies in Mumbai.
Featured by The Manifest among the Top 30 IT Services Company in Mumbai and listed at #7 in the Top 20 BI App Developers in Mumbai.
Leading the list on Cision PRWeb's Top Software Development Companies of 2020.
Ranked at #4 by Visual Objects on Top IT Consulting Firms in Mumbai
Recognized by Great Managers' Forbes India.
""",return_tensors="pt").input_ids


In [65]:
peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

'Big Rattle Technologies'